<center>
<figure>
  <img src="https://raw.githubusercontent.com/sanjaynagi/ax-vampir-hackathon/refs/heads/main/docs/new-vigg-logo1.png" alt="Description" width="480" height="180">
</figure>
</center>


## **Ax-vampIR VIGG Hackathon: Designing amplicon panels in other disease vectors**

Targeted genomic surveillance through amplicon sequencing allows us
to focus on specific genomic regions of interest, enabling routine surveillance at larger scale than whole-genome sequencing. For vectors,
this enables the tracking of insecticide resistance, species
ID markers, and other important genetic variants.

#### **Workshop Aims**

1.  Find orthologous insecticide resistance loci in your target vector
    species using IRTHO
2.  Design a multiplexed PCR primer panel targeting these loci using
    MULTIPLY
<br></br>



### Ag-vampIR and AmpSeeker recap

Ag-vampIR (Anopheles gambiae Vector Amplicon Marker Panel for
Insecticide Resistance) is a targeted sequencing panel designed for
genomic surveillance of malaria vectors. The panel targets 90 important
genomic loci across 80 amplicons in the Anopheles gambiae genome,
including:

-   55 insecticide resistance-associated SNPs
    -   Target-site mutations (kdr, Ace1, Rdl)
    -   Metabolic resistance markers
    -   SNPs tagging selective sweeps
-   35 ancestry informative markers (AIMs) for species identification
-   Two amplicons targeting the doublesex gene drive locus

Each amplicon is approximately 200 bp long. The protocols design allows
multiplexing of up to 1,536 samples on a single flow cell using dual
indexing with 96 i7 and 16 i5 adaptors.

#### **AmpSeeker**

AmpSeeker is an open-source computational pipeline designed for
reproducible analysis of amplicon sequencing data, which can analyze
data from any Illumina amplicon sequencing panel.

<center>
<figure>
  <img src="https://raw.githubusercontent.com/sanjaynagi/ax-vampir-hackathon/refs/heads/main/docs/figure1_new.png" alt="Description" width="600" height="600">
  <figcaption>Figure: A) Ag-vampIR loci B) Overview of the Ag-vampIR laboratory protocol and AmpSeeker bioinformatics pipeline. </figcaption>
</figure>
</center>

<br></br>


## Part 1: Introduction to Irtho

Many known mutations that confer insecticide resistance in insects are examples of parallel evolution; they are shared between species. Although you would ideally have a set of resistance markers to target in your organism of choice, exploiting the likelihood of parallel evolution can help us design panels in other, lesser known vectors. The aim of the first part of this workshop is take known insecticide resistance loci in one vector, and find the orthologous regions in another.

Irtho is a Python package that identifies orthologous loci between
reference genomes. It uses [OrthoFinder](https://github.com/davidemms/OrthoFinder) to find orthologous genes between reference genomes, and provides tools to support the analysis.

Key capabilities:
- Identifies orthologous genes between species   
- Determines synteny between orthologous genes   
- Aligns protein sequences to find the orthologous positions of amino acid residues
- Suggests residues to target in case of no prior knowledge

![IRTHO Workflow](https://placeholder-for-irtho-workflow.png) *Figure 2:
IRTHO workflow diagram*

### Available Reference Genomes

Currently available in pre-computed OrthoFinder results:
- Anopheles gambiae (PEST)
- Anopheles stephensi (UCISS2018)
- Anopheles dirus (WRAIR2)
- Anopheles minimus (MINIMUS2)
- Anopheles sinensis (CHINA)
- Aedes aegypti (LVP_AGWG)   
- Aedes albopictus (AalbF5)
- Culex quinquefasciatus (JHB2020)
- Lutzomyia Longipalpis (ASM)
- Additional vectors can be added using protein FASTA and GFF files from
VectorBase or RefSeq

For the hackathon I suggest sticking to the pre-computed results, although i note that OrthoFinder is super fast.



## Part 2: Introduction to Multiply

Multiply is a command-line tool for designing multiplexed PCR primers.
It was recently published in [Nature Comms](https://www.nature.com/articles/s41467-024-45688-z) alongside NOMADS8 and NOMADS16, nanopore panels for *Plasmodium falciparum* and supports both Illumina and Nanopore.

<center>
<figure>
  <img src="https://raw.githubusercontent.com/JasonAHendry/multiply/master/.images/multiply-pipeline.png" alt="Description" width="800" height="600">
  <figcaption>MULTIPLY pipeline overview</figcaption>
</figure>
</center>

### Design Considerations

When designing amplicon panels, it is important to consider these
factors:

**Sequencing Platforms**  

This workshop can be used to design amplicon panels for both Illumina
and Oxford Nanopore Technologies (ONT) sequencing. Illumina enables use
of the AmpSeeker pipeline and is consistent with the ongoing training in
the EAVES and WAVES projects. The only modification needed for Nanopore
would be adjusting amplicon size in MULTIPLY.

**Amplicon Size**  
For Illumina sequencing, aim for ~200bp to enable
complete coverage with paired-end reads and maximize sequencing
efficiency. For Nanopore, amplicons can be 400-3000bp.

**Primer Specificity**  
Design primers 18-22 bases long with 40-60% GC
content and similar melting temperatures (57-63°C) to ensure specific
amplification of target regions.

**Genetic Variation**  
Check existing population genomic data to avoid
placing primers over known polymorphic sites, which can cause allele
dropout and false negatives. Multiply can take a BED file of SNP
locations and consider this when selecting optimal primers.

**Multiplexing Compatibility**  
Avoid complementary sequences between
primers that could form dimers, and aim for similar GC content and
amplicon sizes. Multiply takes care of this.

---

# **Workshop**

#### **Install prerequisities**

In [ ]:
# Clone required repositories
!git clone https://github.com/sanjaynagi/ax-vampir-hackathon
!git clone https://github.com/sanjaynagi/irtho
!git clone https://github.com/sanjaynagi/multiply # must use my fork

# get bedtools if performing snp analysis
# !wget https://github.com/arq5x/bedtools2/releases/download/v2.29.1/bedtools-2.29.1.tar.gz
# !tar -zxvf bedtools-2.29.1.tar.gz
# !cd bedtools2 && make && cp bin/bedtools /bin/

# get primer3-core binary
!git clone https://github.com/primer3-org/primer3.git primer3
!cd primer3/src && make
!cp primer3/src/primer3_core /bin/

# get blast binaries
!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.16.0+-x64-linux.tar.gz
!tar -xvzf ncbi-blast-2.16.0+-x64-linux.tar.gz
!cp ncbi-blast-2.16.0+/bin/* /bin/

In [ ]:
# install python dependencies, multiply does not specify them
%pip install -e /content/multiply /content/irtho primer3-py pandas numpy numba pysam seaborn biopython scikit-allel -q

In [ ]:
import irtho
import allel
import multiply
import pandas as pd

### Part 1: Finding Orthologous Loci with Irtho

In [ ]:
# set ref and target species (ref must be AgambiaePEST for now unless different targets_df used)
reference_species = "AgambiaePEST"
target_species = "LlongipalpisASM"

results_dir = "/content/ax-vampir-hackathon/results/orthofinder/Results_Feb03_1"
reference_dir = "/content/ax-vampir-hackathon/resources/reference"
!mkdir ax-vampir-hackathon/resources/reference/

# Reference and target species downloads
!wget https://vectorbase.org/common/downloads/release-68/{reference_species}/fasta/data/VectorBase-68_{reference_species}_AnnotatedProteins.fasta -O ax-vampir-hackathon/resources/reference/{reference_species}_AnnotatedProteins.fasta
!wget https://vectorbase.org/common/downloads/release-68/{reference_species}/gff/data/VectorBase-68_{reference_species}.gff -O ax-vampir-hackathon/resources/reference/{reference_species}.gff
!wget https://vectorbase.org/common/downloads/release-68/{target_species}/fasta/data/VectorBase-68_{target_species}_AnnotatedProteins.fasta -O ax-vampir-hackathon/resources/reference/{target_species}_AnnotatedProteins.fasta
!wget https://vectorbase.org/common/downloads/release-68/{target_species}/gff/data/VectorBase-68_{target_species}.gff -O ax-vampir-hackathon/resources/reference/{target_species}.gff

# refseq download
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/024/334/085/GCF_024334085.1_ASM2433408v1/GCF_024334085.1_ASM2433408v1_genomic.gff.gz -O ax-vampir-hackathon/resources/reference/{target_species}.gff.gz
# !gzip -d -f ax-vampir-hackathon/resources/reference/{target_species}.gff.gz
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/024/334/085/GCF_024334085.1_ASM2433408v1/GCF_024334085.1_ASM2433408v1_protein.faa.gz -O ax-vampir-hackathon/resources/reference/{target_species}_AnnotatedProteins.fasta.gz
# !gzip -d -f ax-vampir-hackathon/resources/reference/{target_species}_AnnotatedProteins.fasta.gz

# load input targets
targets_df = pd.read_csv("/content/ax-vampir-hackathon/resources/irtho-targets.tsv", sep="\t")
targets_df

In [ ]:
# create orthologs results object
ortho = irtho.Orthologs(results_dir)

# find orthologs in your species and add to dataframe
targets_ortho_df = ortho.map_input_genes_to_orthologs(
    targets_df,
    reference_species,
    target_species
)

# split one to many orthologs to different rows
targets_ortho_df = irtho.split_one_to_many_orthologs(targets_ortho_df, target_species)

# determine synteny
targets_ortho_df = irtho.add_synteny_information(
    targets_ortho_df,
    reference_dir=reference_dir,
    reference_species=reference_species,
    target_species=target_species,
    ortho_obj=ortho
)

In [ ]:
targets_ortho_df

In [ ]:
# find genomic coordinates of equivalent codons (where we provide codon info in reference)
targets_ortho_df = ortho.find_orthologous_targets(
    targets_ortho_df,
    reference_dir=reference_dir,
    ref_genome=reference_species,
    target_genome=target_species
)

final_targets_df = ortho.process_random_targets(
    targets_ortho_df,
    reference_dir=reference_dir,
    target_species=target_species
)

final_targets_df

In [ ]:
# write targets with coordinates to BED format + unique id column
final_targets_bed = final_targets_df.assign(unique_id=lambda x: x[target_species] + "-" + x.codon.astype(str))[['target_contig', 'target_start', 'target_end', 'unique_id']][final_targets_df['target_start'].astype(str).str.isnumeric()]
final_targets_bed = final_targets_bed.drop_duplicates()
final_targets_bed.to_csv("targets.bed", sep="\t", index=False, header=False)
final_targets_bed

### Part 2: Designing Primers with MULTIPLY

First, edit the multiply/genomes/collection.ini and add your vector species! then run the below download command.

In [ ]:
# download reference genome for your target species
!multiply download -g {target_species}

In [ ]:
# regions in proximity of less than the amplicon length must first be merged or multiply will fail
!python ax-vampir-hackathon/merge-bed.py targets.bed targets_merged.bed -d 130

### Optional - add variant information

Multiply can take SNP data in BED format to determine if SNPs exist in primer binding sites. Here is a [notebook to output a bed file from malariagen_data](https://colab.research.google.com/drive/1k9Cox3OGiCR59KcNblsWOGCv2mUkVDuI?usp=sharing). In this case, you must include a flag in the `collection.ini` which provides a path to a json. For example:

`include_variation = genomes/variation/PlasmodiumFalciparum/files.json`


### Run multiply

Now, we must edit the design .ini file in ax-vampir-hackathon directory, to fit your species and preferences

In [ ]:
# run MULTIPLY pipeline
!multiply pipeline -d ax-vampir-hackathon/ax-vampir.ini

### Panel design complete!!

Download the results folder from the left panel!!

## Resources

-   irtho Documentation: [GitHub](https://github.com/sanjaynagi/irtho)
-   multiply Documentation: [GitHub](https://github.com/JasonAHendry/multiply)
-   multiply Paper: [Nature Communications](https://www.nature.com/articles/s41467-024-45688-z)
-   VectorBase: <https://vectorbase.org>